# JSON parser - Tutorial

## Creating the Parser

In [1]:
from lark import Lark
json_parser = Lark(r"""
    value: dict
         | list
         | ESCAPED_STRING
         | SIGNED_NUMBER
         | "true" | "false" | "null"

    list : "[" [value ("," value)*] "]"

    dict : "{" [pair ("," pair)*] "}"
    pair : ESCAPED_STRING ":" value

    %import common.ESCAPED_STRING
    %import common.SIGNED_NUMBER
    %import common.WS
    %ignore WS

    """, start='value')

In [2]:
text = '{"key": ["item0", "item1", 3.14]}'
json_parser.parse(text)

Tree(Token('RULE', 'value'), [Tree(Token('RULE', 'dict'), [Tree(Token('RULE', 'pair'), [Token('ESCAPED_STRING', '"key"'), Tree(Token('RULE', 'value'), [Tree(Token('RULE', 'list'), [Tree(Token('RULE', 'value'), [Token('ESCAPED_STRING', '"item0"')]), Tree(Token('RULE', 'value'), [Token('ESCAPED_STRING', '"item1"')]), Tree(Token('RULE', 'value'), [Token('SIGNED_NUMBER', '3.14')])])])])])])

In [3]:
print( _.pretty() )

value
  dict
    pair
      "key"
      value
        list
          value	"item0"
          value	"item1"
          value	3.14

